In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import xarray
import pyresample #use pyresample package to resample SSH into uniform grid
import glob
from netCDF4 import Dataset

/vortexfs1/home/wwu/.conda/envs/myenv/lib/python3.8/site-packages/pyresample/bilinear/__init__.py:50: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


In [6]:
nc =  Dataset('/vortexfs1/home/wwu/Rossby_Wave_Project/data/pacific_ssh_26_39.nc','r')
#nc =  Dataset('data/pacific_ssh_26_39.nc','r')
lon = nc.variables['lon2d'][:] # non-uniform longitude in NCAR model output
lat = nc.variables['lat2d'][:] # non-uniform latitude
h = nc.variables['ssh'][:] # 3D ssh field (14 years) from NCAR model

In [8]:
orig_def = pyresample.geometry.SwathDefinition(lons=lon, \
                                               lats=lat)
lon2 = np.append(np.arange(120-1.5,180.1,0.1),\
                np.arange(-179.9,-69.9+1.5,0.1))
lat2 = np.arange(-30-1.5,60.1+1.5,0.1)
lon_tar, lat_tar = np.meshgrid(lon2, lat2) # target uniform 0.1 degree resolution grid
targ_def = pyresample.geometry.SwathDefinition(lons=lon_tar, \
                                               lats=lat_tar)

In [9]:
window = 1./31. * np.ones(31,) # 3degree by 3degree running average box
ssh_resample = np.ma.masked_all([1022,931,1731])
ssh_filter_y =np.ma.masked_all([1022,901,1731])
ssh_filter_xy =np.ma.masked_all([1022,901,1701])

for i in range(1022):
    print(i)
    h2 = pyresample.kd_tree.resample_gauss(orig_def, h[i,:,:],\
     targ_def, neighbours=9, radius_of_influence=2e4, sigmas=1e4,fill_value=None)
    ssh_resample[i,:,:] = h2 # h2 is 0.1 degree resolution SSH in uniform grid
    
    for x in range(1731):
        ssh_filter_y[i,:,x]=np.convolve(h2[:,x],window,mode = 'valid')
    for y in range(901):
        h3 = np.convolve(ssh_filter_y[i,y,:],window,mode = 'valid')
        ssh_filter_xy[i,y,:] = np.ma.masked_where(h3>1e3,h3) # Mask Land Grid 
        #ssh_filter_xy is the 3degree by 3degree filtered SSH
     

0


/vortexfs1/home/wwu/.conda/envs/myenv/lib/python3.8/site-packages/pyresample/kd_tree.py:385: UserWarning: Possible more than 9 neighbours within 20000.0 m for some data points
  warnings.warn(('Possible more than %s neighbours '


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

In [ ]:
#save data as nc.file

#ncout = Dataset('pacific_resample_ssh_26_39.nc','w','NETCDF4');
#ncout.createDimension('lat',901)
#ncout.createDimension('lon',1701)
#ncout.createDimension('time',1022)

#lat1 = ncout.createVariable('lat','float32',('lat'))
#lat1[:] = lat2[15:-15]
#lon1 = ncout.createVariable('lon','float32',('lon'))
#lon1[:] = lon2[15:-15]

#ssh1 = ncout.createVariable('ssh_resample','float32',('time','lat','lon'))
#ssh1[:] = ssh_resample[:,15:-15,15:-15]

#ssh2 = ncout.createVariable('ssh_filter','float32',('time','lat','lon'))
#ssh2[:] = ssh_filter_xy

#ncout.close()

In [4]:
#how to use 'pacific_resample_ssh_26_39.nc' to produce "fft_filtered_hann_detrended_all_freqs.nc"
## note: it takes about a day to run this cell

N = 1022
dt = 5.#days
f_n = 1/(2*dt)
f = np.linspace(0.0, f_n, num=int((N)/2)+1)
df = f[1] - f[0]
nc1 = Dataset('/vortexfs1/home/wwu/Rossby_Wave_Project/data/pacific_resample_ssh_26_39.nc')
#nc1 = Dataset('data/pacific_resample_ssh_26_39.nc')

lat2 = nc1.variables['lat'][:]
lon2 = nc1.variables['lon'][:]
lon2,lat2 = np.meshgrid(lon2,lat2)
lon2=360*(lon2<0)+lon2

w = np.sqrt(8./3.)
fft_h_fl =(0.+0j)*np.ma.masked_all_like(np.zeros((512,901,1701)))
for nlat in range(901):
    for nlon in range(1701):
        if np.ma.is_masked(nc1.variables['h_filter'][:,nlat,nlon]) == False:
            h = w * np.hanning(1022) * signal.detrend(nc1.variables['h_filter'][:,nlat,nlon])
            fft_h_fl[:,nlat,nlon] = np.fft.rfft(h)

In [ ]:
#ncout = Dataset('fft_filtered_hann_detrend_ssh_14yrs_all_freqs.nc','w','NETCDF4');
#ncout.createDimension('lat',901)
#ncout.createDimension('lon',1701)
#ncout.createDimension('frequency',512)
#
#latvar = ncout.createVariable('lat','float32',('lat'))
#latvar[:] = nc2.variables['lat'][:]
#
#lonvar = ncout.createVariable('lon','float32',('lon'))
#lonvar[:] = nc2.variables['lon'][:]
#
#cpd = ncout.createVariable('cycle_per_day','float32',('frequency'))
#cpd[:] = f[:]
#
#
#fftreal2 = ncout.createVariable('fft_h_filtered_real','float64',('frequency','lat','lon'))
#fftreal2[:] = np.real(fft_h_fl)
#fftimg2 = ncout.createVariable('fft_h_filtered_img','float64',('frequency','lat','lon'))
#fftimg2[:] = np.imag(fft_h_fl)
#
#
#ncout.close()